In [1]:
# Import useful packages
import os
import time
import pandas as pd
import glob
import numpy as np

# For loading/querying postgres
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import connect

In [2]:
# Create SQL engine to load data to postgres for central usage
from sqlalchemy import create_engine
dbname = '697_temp'
user = 'postgres'
password = 'poRter!5067'
host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com'

engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

### Loading Open Positions to Postgres

In [37]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Open_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Open_Positions_Data/RF Reg_target_60_rebal_30_2017-01-01.csv',
 'Open_Positions_Data/RF Reg_target_120_rebal_30_2017-01-01.csv',
 'Open_Positions_Data/RF Reg_target_120_rebal_60_2017-01-01.csv']

In [38]:
# create temp_df to test loading; pick model not already in table
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.head()

,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
0,Positions_2020-08-06,2020-08-06,PEAK,1407,37110.152424,26.375375,37110.152424,26.375375,0.0,RF Reg_target_60_rebal_30_2017-01-01
1,Positions_2020-08-06,2020-08-06,DOW,132,5251.401484,39.783345,5251.401484,39.783345,0.0,RF Reg_target_60_rebal_30_2017-01-01
2,Positions_2020-08-06,2020-08-06,MO,116,4476.208144,38.588001,4476.208144,38.588001,0.0,RF Reg_target_60_rebal_30_2017-01-01
3,Positions_2020-08-06,2020-08-06,KIM,395,4340.217862,10.987893,4340.217862,10.987893,0.0,RF Reg_target_60_rebal_30_2017-01-01
4,Positions_2020-08-06,2020-08-06,C,80,3958.858682,49.485734,3958.858682,49.485734,0.0,RF Reg_target_60_rebal_30_2017-01-01


In [40]:
temp_df.columns

Index(['key', 'Date', 'Ticker', 'Quantity', 'Basis', 'Purchase Price',
       'Current Value', 'Last', '% Gain', 'model'],
      dtype='object')

In [21]:
# Initial dataframe needs to create a new table
temp_df.to_sql('open_positions', engine)

In [22]:
# Append data to table
temp_df.to_sql('open_positions', engine, if_exists='append')

In [24]:
# Test with a load
def import_open_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM open_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['key', 'model'])
    return data
open_pos_df = import_open_positions().drop('index',axis=1)

open_pos_df.tail()


,key,Date,Ticker,Quantity,Basis,Purchase Price,Current Value,Last,% Gain,model
12190,Positions_2021-03-03,2020-08-06,OXY,76,1166.702928,15.351354,2166.619136,28.508147,85.70,RF Reg_target_60_rebal_30_2017-01-01
12191,Positions_2021-03-03,2020-08-06,TWTR,26,967.460001,37.210000,1885.195042,72.507502,94.86,RF Reg_target_60_rebal_30_2017-01-01
12192,Positions_2021-03-03,2020-08-06,MRK,11,826.829912,75.166356,753.771087,68.524644,-8.84,RF Reg_target_60_rebal_30_2017-01-01
12193,Positions_2021-03-03,2020-08-06,CMCSA,19,795.295593,41.857663,1021.744903,53.776048,28.47,RF Reg_target_60_rebal_30_2017-01-01
12194,Positions_2021-03-03,2020-09-04,MRO,7,31.318420,4.474060,80.987188,11.569598,158.59,RF Reg_target_60_rebal_30_2017-01-01


### Loading track records to postgres

In [25]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("track_record/*.csv"):
    txtfiles.append(file)
    
txtfiles

['track_record/RF Reg_target_60_rebal_30_2017-01-01.csv',
 'track_record/RF Reg_target_120_rebal_30_2017-01-01.csv',
 'track_record/RF Reg_target_120_rebal_60_2017-01-01.csv']

In [26]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.head()

,Date,Value,Val_ex_cash,model
0,2020-08-06,100000.000000,99465.751610,RF Reg_target_60_rebal_30_2017-01-01
1,2020-08-07,101398.918613,100864.670223,RF Reg_target_60_rebal_30_2017-01-01
2,2020-08-10,103963.916785,103429.668394,RF Reg_target_60_rebal_30_2017-01-01
3,2020-08-11,105306.857052,104772.608661,RF Reg_target_60_rebal_30_2017-01-01
4,2020-08-12,104401.500965,103867.252575,RF Reg_target_60_rebal_30_2017-01-01


In [ ]:
# Initial dataframe needs to create a new table
temp_df.to_sql('track_record', engine)

In [27]:
# Append data to table
temp_df.to_sql('track_record', engine, if_exists='append')

In [28]:
# Test with a load
def import_track_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM track_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['Date', 'model'])
    return data
track_df = import_track_record().drop('index',axis=1)

track_df.head()


,Date,Value,Val_ex_cash,model
144,2020-07-19 00:00:00,100000.0000,100000.0000,75percent_confidence_no_holding_15m_cnn
145,2020-07-20 00:00:00,102126.2298,102126.2298,75percent_confidence_no_holding_15m_cnn
146,2020-07-21 00:00:00,101447.4265,101447.4265,75percent_confidence_no_holding_15m_cnn
147,2020-07-22 00:00:00,102702.7938,102702.7938,75percent_confidence_no_holding_15m_cnn
148,2020-07-23 00:00:00,102327.8937,102327.8937,75percent_confidence_no_holding_15m_cnn


### Loading cash records to postgres

In [29]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Cash_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Cash_Positions_Data/RF Reg_target_60_rebal_30_2017-01-01.csv',
 'Cash_Positions_Data/RF Reg_target_120_rebal_30_2017-01-01.csv',
 'Cash_Positions_Data/RF Reg_target_120_rebal_60_2017-01-01.csv']

In [30]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.tail()

,key,cash,model
139,cash_2021-02-25,23.770446,RF Reg_target_60_rebal_30_2017-01-01
140,cash_2021-02-26,23.770446,RF Reg_target_60_rebal_30_2017-01-01
141,cash_2021-03-01,23.770446,RF Reg_target_60_rebal_30_2017-01-01
142,cash_2021-03-02,23.770446,RF Reg_target_60_rebal_30_2017-01-01
143,cash_2021-03-03,23.770446,RF Reg_target_60_rebal_30_2017-01-01


In [31]:
# Initial dataframe needs to create a new table
temp_df.to_sql('cash_record', engine, if_exists='append')

In [32]:
# Test with a load
def import_cash_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM cash_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
cash_df = import_cash_record().drop('index',axis=1)

cash_df.tail()


,key,cash,model
777,cash_2021-02-25,23.770446,RF Reg_target_60_rebal_30_2017-01-01
778,cash_2021-02-26,23.770446,RF Reg_target_60_rebal_30_2017-01-01
779,cash_2021-03-01,23.770446,RF Reg_target_60_rebal_30_2017-01-01
780,cash_2021-03-02,23.770446,RF Reg_target_60_rebal_30_2017-01-01
781,cash_2021-03-03,23.770446,RF Reg_target_60_rebal_30_2017-01-01


In [13]:
cash_df.count()

key      268
cash     268
model    268
dtype: int64

### Loading Value Positions to postgres

In [33]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Val_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Val_Positions_Data/RF Reg_target_60_rebal_30_2017-01-01.csv',
 'Val_Positions_Data/RF Reg_target_120_rebal_30_2017-01-01.csv',
 'Val_Positions_Data/RF Reg_target_120_rebal_60_2017-01-01.csv']

In [34]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.tail()

,key,value,model
139,val_2021-02-25,131991.759044,RF Reg_target_60_rebal_30_2017-01-01
140,val_2021-02-26,128376.205474,RF Reg_target_60_rebal_30_2017-01-01
141,val_2021-03-01,129952.513408,RF Reg_target_60_rebal_30_2017-01-01
142,val_2021-03-02,130075.125108,RF Reg_target_60_rebal_30_2017-01-01
143,val_2021-03-03,130563.511682,RF Reg_target_60_rebal_30_2017-01-01


In [35]:
# Initial dataframe needs to create a new table
temp_df.to_sql('val_positions', engine,if_exists='append')

In [36]:
# Test with a load
def import_val_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM val_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
val_df = import_val_positions().drop('index',axis=1)

val_df.tail()

,key,value,model
777,val_2021-02-25,131991.759044,RF Reg_target_60_rebal_30_2017-01-01
778,val_2021-02-26,128376.205474,RF Reg_target_60_rebal_30_2017-01-01
779,val_2021-03-01,129952.513408,RF Reg_target_60_rebal_30_2017-01-01
780,val_2021-03-02,130075.125108,RF Reg_target_60_rebal_30_2017-01-01
781,val_2021-03-03,130563.511682,RF Reg_target_60_rebal_30_2017-01-01
